In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize

Using TensorFlow backend.


[10:08:28] DEBUG Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.


In [3]:
# -*- coding: utf-8 -*-
# @File  : 中文文本分类20200609-4分类版本.ipynb
# @Author: 英俊
# @Date  : 2020/6/9
# @Software: jupyter notebook
# 复旦大学语料文本分类

import pandas as pd

import jieba

import os

import pickle  # 持久化

from numpy import *

#数据提取
import sklearn

from sklearn.feature_extraction.text import TfidfTransformer  # TF-IDF向量转换类

from sklearn.feature_extraction.text import TfidfVectorizer  # TF_IDF向量生成类


#模型建立
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier,AdaBoostClassifier
from sklearn.svm import SVC,LinearSVC
from sklearn.naive_bayes import BernoulliNB,MultinomialNB,GaussianNB
from sklearn.neighbors import KNeighborsClassifier
import xgboost
from xgboost import XGBClassifier
# 集成模块
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier,AdaBoostClassifier
from sklearn.ensemble import RandomForestRegressor,BaggingRegressor,AdaBoostRegressor
from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestRegressor,BaggingRegressor,AdaBoostRegressor

#Pipeline 使用一系列 (key, value) 键值对来构建,其中 key 是你给这个步骤起的名字， value 是一个评估器对象:
from sklearn.pipeline import Pipeline
#准确率，精确率，召回率，f1
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,classification_report
# 保存数据
import joblib

# 读取数据
def readFile(path):
    with open(path, 'r', errors='ignore') as file:  # 文档中编码有些问题，所有用errors过滤错误
        content = file.read()
        return content

# 这块没用，供日后研究一下
# 保存文件
def saveFile(path, result):
    with open(path, 'w', errors='ignore') as file:
        file.write(result)

In [4]:
import jieba

import os

import pickle  # 持久化

from numpy import *

In [5]:
def get_data():
    resultList=[]
    labeList=[]
    inputPath="./data/"
    fatherLists = os.listdir(inputPath)  # 主目录
    for eachDir in fatherLists:  # 遍历主目录中各个文件夹
        print(eachDir)
        eachPath = inputPath + eachDir + "/"  # 保存主目录中每个文件夹目录，便于遍历二级文件
        print(eachPath)
        childLists = os.listdir(eachPath)  # 获取每个文件夹中的各个文件
    #     print(eachPath)
    #     each_resultPath = resultPath + eachDir + "/"  # 分词结果文件存入的目录
        for eachFile in childLists:  # 遍历每个文件夹中的子文件
                    eachPathFile = eachPath + eachFile  # 获得每个文件路径
                    content = readFile(eachPathFile)  # 调用上面函数读取内容
                    result = (str(content)).replace("\r\n", "").strip()  # 删除多余空行与空格
                    result = content.replace("\r\n","").strip()
                    labeList.append(eachDir)
                    resultList.append(result)
                    cutResult = jieba.cut(result)  # 默认方式分词，分词结果用空格隔开
    return labeList,resultList

In [6]:
labeList,resultList=get_data()

C3-Art
./data/C3-Art/
C4-Literature
./data/C4-Literature/
C5-Education
./data/C5-Education/
C6-Philosophy
./data/C6-Philosophy/


In [7]:
print(len(labeList))
print(len(resultList))

825
825


In [8]:
dictTmp={ "label":labeList ,"result":resultList}
df_all=pd.DataFrame(dictTmp)
df_all.head()

,label,result
0,C3-Art,【 文献号 】1-2340\n【原文出处】中国图书评论\n【原刊地名】沈阳\n【原刊期号】1...
1,C3-Art,【 文献号 】1-2681\n【原文出处】武汉大学学报：哲社版\n【原刊期号】199605\...
2,C3-Art,【 文献号 】2-4116\n【原文出处】云南学术探索\n【原刊地名】昆明\n【原刊期号】1...
3,C3-Art,【 文献号 】1-3036\n【原文出处】文艺理论研究\n【原刊地名】沪\n【原刊期号】19...
4,C3-Art,【 文献号 】1-3066\n【原文出处】文学评论\n【原刊地名】京\n【原刊期号】1995...


In [9]:
df_all["result"][0]

'【 文献号 】1-2340\n【原文出处】中国图书评论\n【原刊地名】沈阳\n【原刊期号】199510\n【原刊页号】61-62\n【分 类 号】Z1\n【分 类 名】出版工作、图书评介\n【 作  者 】杨小民\n【复印期号】199602\n【 标  题 】图书评论应当重视对书籍装帧艺术的评价\n【 正  文 】\n    图书评论是近代报刊业兴起后，在世界各国得到长足发展的一种新型评论体裁。而不论是书评理论还是书评实践都有一个不小的疏漏，即忽\n视了图书的形式因素。因为图书是内容与形式的综合体，忽视了“图书形式”这一重要方面，会导致在图书评论活动中忽视对图书的出版形式这\n一重要方面的品评论述，而这对于出版物的达到基本要求：“形神俱佳”（“形”指书装艺术，“神”指内容叙述）或最高要求“尽善尽美”（\n“尽善”指内容而言，“尽美”指形式而言）无疑是有缺憾的。\n    图书的形式因素即为书籍的装帧设计艺术（以下简称“书装艺术”）。它的内容应当包括：封面、封底、书脊、环衬、扉页、字体、字号、\n插图、版式、护封等。装帧设计应是图书中的重要内容，顺理成章地应成为书评文章中不可或缺的评论对象。然而，在当前报刊上大量刊登的书\n评文章中谈及这一方面的极为少见。这一偏颇势必会对中国出版物综合水平的提高产生不良的影响。\n    图书出版事业是人类的思维活动和精神成果与科学技术相结合的一项系统工程。而书装艺术则渗透着“出版人”的思维活动和印刷科技的水\n平两个因素。设计者的艺术构思，通过印刷工艺的精心制作，与图书的内容达到协调一致，才形成一本精美的形神俱佳的图书。\n    如今，我国的一些出版社，对图书的装帧设计重视不够，这既成为书评作者忽视书装艺术的评论的一个潜因，他们认为许多图书的书装艺术\n不值一提或难以一说；同时，也人为地造成了对书装艺术粗糙现象的不合理宽容。究其原因，出版社不愿投入应有的资金和人力是主要问题。书\n装艺术本身也是体现出版物品位高低的一项重要因素。在现代图书出版印刷中，应投入必要的资金，以避免参加国际图书博览会的中国图书再被\n人们讥笑为“展翅高飞”、“鞠躬尽瘁”了。（由于纸质差，装订落后，我国图书陈列于国际展台时，暖气会使书册张开弯曲，这叫“展翅高飞\n”；还有则为书脊软塌，不能直立，弯腰驼背，则称“鞠躬尽瘁”。）\n    编辑素养的欠缺，也直

In [10]:
def remove_text(text):
    text=text.replace('\n','')
    text=text.replace(' ','')
    text=text.replace('【','')
    text=text.replace('】','')
    text=text.replace('（','')
    text=text.replace('）','')
    text=text.replace('、','')
    text=text.replace('。','')
    text=text.replace('；','')
    text=text.replace('“','')
    text=text.replace('”','')
    text=text.replace('…','')
    text=text.replace('，','')
    text=text.replace('：','')
    text=text.replace('《','')
    text=text.replace('》','')
    text=text.replace('[','')
    text=text.replace(']','')
    text=text.replace('—','')
    return text

In [11]:
txt=remove_text(df_all["result"][0])
print(txt)

文献号1-2340原文出处中国图书评论原刊地名沈阳原刊期号199510原刊页号61-62分类号Z1分类名出版工作图书评介作者杨小民复印期号199602标题图书评论应当重视对书籍装帧艺术的评价正文图书评论是近代报刊业兴起后在世界各国得到长足发展的一种新型评论体裁而不论是书评理论还是书评实践都有一个不小的疏漏即忽视了图书的形式因素因为图书是内容与形式的综合体忽视了图书形式这一重要方面会导致在图书评论活动中忽视对图书的出版形式这一重要方面的品评论述而这对于出版物的达到基本要求形神俱佳形指书装艺术神指内容叙述或最高要求尽善尽美尽善指内容而言尽美指形式而言无疑是有缺憾的图书的形式因素即为书籍的装帧设计艺术以下简称书装艺术它的内容应当包括封面封底书脊环衬扉页字体字号插图版式护封等装帧设计应是图书中的重要内容顺理成章地应成为书评文章中不可或缺的评论对象然而在当前报刊上大量刊登的书评文章中谈及这一方面的极为少见这一偏颇势必会对中国出版物综合水平的提高产生不良的影响图书出版事业是人类的思维活动和精神成果与科学技术相结合的一项系统工程而书装艺术则渗透着出版人的思维活动和印刷科技的水平两个因素设计者的艺术构思通过印刷工艺的精心制作与图书的内容达到协调一致才形成一本精美的形神俱佳的图书如今我国的一些出版社对图书的装帧设计重视不够这既成为书评作者忽视书装艺术的评论的一个潜因他们认为许多图书的书装艺术不值一提或难以一说同时也人为地造成了对书装艺术粗糙现象的不合理宽容究其原因出版社不愿投入应有的资金和人力是主要问题书装艺术本身也是体现出版物品位高低的一项重要因素在现代图书出版印刷中应投入必要的资金以避免参加国际图书博览会的中国图书再被人们讥笑为展翅高飞鞠躬尽瘁了由于纸质差装订落后我国图书陈列于国际展台时暖气会使书册张开弯曲这叫展翅高飞还有则为书脊软塌不能直立弯腰驼背则称鞠躬尽瘁编辑素养的欠缺也直接影响到书装艺术的优劣在我国的出版业中编辑通常是提供书装要求并参与设计方案的参与的前提应该是要具备一定的艺术素质和审美眼光但如今有相当一部分编辑缺乏这一点他们对艺术规律对美术设计者从事的工作特性知之甚少他们的参与从某种意义上来说甚至成为一种盲目的干涉外行指挥内行大至约束个框子小至书名作者的位置安放和颜色的指派不难设想在这种缺乏平等探讨的格局下要求所设计出来的封扉等的艺术效果将是什么样子当然提出这些问题并不

In [12]:
def multiclass_logloss(actual, predicted, eps=1e-15):
    """对数损失度量（Logarithmic Loss  Metric）的多分类版本。
    :param actual: 包含actual target classes的数组
    :param predicted: 分类预测结果矩阵, 每个类别都有一个概率
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

In [13]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
x=[]
encoder_x=LabelEncoder()
x=encoder_x.fit_transform(df_all['label'])
# print(lelist)
df_all['label_pred']=x
df_all.head()

,label,result,label_pred
0,C3-Art,【 文献号 】1-2340\n【原文出处】中国图书评论\n【原刊地名】沈阳\n【原刊期号】1...,0
1,C3-Art,【 文献号 】1-2681\n【原文出处】武汉大学学报：哲社版\n【原刊期号】199605\...,0
2,C3-Art,【 文献号 】2-4116\n【原文出处】云南学术探索\n【原刊地名】昆明\n【原刊期号】1...,0
3,C3-Art,【 文献号 】1-3036\n【原文出处】文艺理论研究\n【原刊地名】沪\n【原刊期号】19...,0
4,C3-Art,【 文献号 】1-3066\n【原文出处】文学评论\n【原刊地名】京\n【原刊期号】1995...,0


In [14]:
import jieba
# jieba.enable_parallel(4) #并行分词开启
df_all['文本分词'] = df_all['result'].apply(lambda i:jieba.cut(i) )
df_all['文本分词'] =[' '.join(i) for i in df_all['文本分词']]

Building prefix dict from the default dictionary ...


[10:12:56] DEBUG Building prefix dict from the default dictionary ...


Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache


[10:12:56] DEBUG Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache


Loading model cost 1.179 seconds.


[10:12:57] DEBUG Loading model cost 1.179 seconds.


Prefix dict has been built successfully.


[10:12:57] DEBUG Prefix dict has been built successfully.


In [15]:
df_all.head()

,label,result,label_pred,文本分词
0,C3-Art,【 文献号 】1-2340\n【原文出处】中国图书评论\n【原刊地名】沈阳\n【原刊期号】1...,0,【 文献号 】 1 - 2340 \n 【 原文 出处 】 中国 图书 评论 \n ...
1,C3-Art,【 文献号 】1-2681\n【原文出处】武汉大学学报：哲社版\n【原刊期号】199605\...,0,【 文献号 】 1 - 2681 \n 【 原文 出处 】 武汉大学 学报 ： 哲社...
2,C3-Art,【 文献号 】2-4116\n【原文出处】云南学术探索\n【原刊地名】昆明\n【原刊期号】1...,0,【 文献号 】 2 - 4116 \n 【 原文 出处 】 云南 学术 探索 \n ...
3,C3-Art,【 文献号 】1-3036\n【原文出处】文艺理论研究\n【原刊地名】沪\n【原刊期号】19...,0,【 文献号 】 1 - 3036 \n 【 原文 出处 】 文艺理论 研究 \n 【...
4,C3-Art,【 文献号 】1-3066\n【原文出处】文学评论\n【原刊地名】京\n【原刊期号】1995...,0,【 文献号 】 1 - 3066 \n 【 原文 出处 】 文学评论 \n 【 原刊...


In [16]:
df_all['result_pred'] =[remove_text(i) for i in df_all['result']]

In [17]:
df_all.head()

,label,result,label_pred,文本分词,result_pred
0,C3-Art,【 文献号 】1-2340\n【原文出处】中国图书评论\n【原刊地名】沈阳\n【原刊期号】1...,0,【 文献号 】 1 - 2340 \n 【 原文 出处 】 中国 图书 评论 \n ...,文献号1-2340原文出处中国图书评论原刊地名沈阳原刊期号199510原刊页号61-62分类...
1,C3-Art,【 文献号 】1-2681\n【原文出处】武汉大学学报：哲社版\n【原刊期号】199605\...,0,【 文献号 】 1 - 2681 \n 【 原文 出处 】 武汉大学 学报 ： 哲社...,文献号1-2681原文出处武汉大学学报哲社版原刊期号199605原刊页号136-137分类号...
2,C3-Art,【 文献号 】2-4116\n【原文出处】云南学术探索\n【原刊地名】昆明\n【原刊期号】1...,0,【 文献号 】 2 - 4116 \n 【 原文 出处 】 云南 学术 探索 \n ...,文献号2-4116原文出处云南学术探索原刊地名昆明原刊期号199706原刊页号77～81分类...
3,C3-Art,【 文献号 】1-3036\n【原文出处】文艺理论研究\n【原刊地名】沪\n【原刊期号】19...,0,【 文献号 】 1 - 3036 \n 【 原文 出处 】 文艺理论 研究 \n 【...,文献号1-3036原文出处文艺理论研究原刊地名沪原刊期号199504原刊页号6-17分类号J...
4,C3-Art,【 文献号 】1-3066\n【原文出处】文学评论\n【原刊地名】京\n【原刊期号】1995...,0,【 文献号 】 1 - 3066 \n 【 原文 出处 】 文学评论 \n 【 原刊...,文献号1-3066原文出处文学评论原刊地名京原刊期号199506原刊页号91-102分类号J...


In [18]:
""" 将所有数字标记映射为一个占位符（Placeholder）。对于许多实际应用场景来说，以数字开头的tokens不是很有用，    但这样tokens的存在也有一定相关性。通过将所有数字都表示成同一个符号，可以达到降维的目的。"""   

def number_normalizer(tokens):    
     return ("#NUMBER" if token[0].isdigit() else token for token in tokens) 

In [19]:
class NumberNormalizingVectorizer(TfidfVectorizer):    
       def build_tokenizer(self):        
            tokenize = super(NumberNormalizingVectorizer, self).build_tokenizer()       
            return lambda doc: list(number_normalizer(tokenize(doc)))

In [20]:
stwlist=[line.strip() for line in open('stopword.txt',
'r',encoding='utf-8').readlines()]
tfv = NumberNormalizingVectorizer(min_df=3,
                                  max_df=0.5,
                                  max_features=None,
                                  ngram_range=(1, 2),
                                  use_idf=True,
                                  smooth_idf=True,
                                  stop_words = stwlist)

In [21]:
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(df_all.label.values)

In [22]:
xtrain, xvalid, ytrain, yvalid = train_test_split(df_all.文本分词.values, y,
                                                  stratify=y,
                                                  random_state=42,
                                                  test_size=0.1, shuffle=True)
print (xtrain.shape)
print (xvalid.shape)

(742,)
(83,)


In [23]:
tfv.fit(list(xtrain) + list(xvalid))
xtrain_tfv =  tfv.transform(xtrain)
xvalid_tfv = tfv.transform(xvalid)

C:\anaconda\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'aren', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'll', 'mon', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [24]:
ctv = CountVectorizer(min_df=3,
                      max_df=0.5,
                      ngram_range=(1,2),
                      stop_words = stwlist)

In [25]:
ctv.fit(list(xtrain) + list(xvalid))
xtrain_ctv =  ctv.transform(xtrain)
xvalid_ctv = ctv.transform(xvalid)

C:\anaconda\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'aren', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'll', 'mon', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [1]:
#创建一个Ensembling主类，具体使用方法见下一个cell
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold
import pandas as pd
import os
import sys
import logging

logging.basicConfig(
    level=logging.DEBUG,
    format="[%(asctime)s] %(levelname)s %(message)s",
    datefmt="%H:%M:%S", stream=sys.stdout)
logger = logging.getLogger(__name__)

In [36]:
# 集成类
class Ensembler(object):
    def __init__(self, model_dict, num_folds=3, task_type='classification', optimize=roc_auc_score,
                 lower_is_better=False, save_path=None):
          """
        Ensembler init function
        :param model_dict: 模型字典
        :param num_folds: ensembling所用的fold数量
        :param task_type: 分类（classification） 还是回归（regression）
        :param optimize: 优化函数，比如 AUC, logloss, F1等，必须有2个函数，即y_test 和 y_pred
        :param lower_is_better: 优化函数（Optimization Function）的值越低越好还是越高越好
        :param save_path: 模型保存路径
        """

        self.model_dict = model_dict
        self.levels = len(self.model_dict)
        self.num_folds = num_folds
        self.task_type = task_type
        self.optimize = optimize
        self.lower_is_better = lower_is_better
        self.save_path = save_path

        self.training_data = None
        self.test_data = None
        self.y = None
        self.lbl_enc = None
        self.y_enc = None
        self.train_prediction_dict = None
        self.test_prediction_dict = None
        self.num_classes = None

    def fit(self, training_data, y, lentrain):
        """
        :param training_data: 二维表格形式的训练数据
        :param y: 二进制的, 多分类或回归
        :return: 用于预测的模型链（Chain of Models）

        """

        self.training_data = training_data
        self.y = y

        if self.task_type == 'classification':
            self.num_classes = len(np.unique(self.y))
logger.info("Found %d classes", self.num_classes)
            self.lbl_enc = LabelEncoder()
            self.y_enc = self.lbl_enc.fit_transform(self.y)
            kf = StratifiedKFold(n_splits=self.num_folds)
            train_prediction_shape = (lentrain, self.num_classes)
        else:
            self.num_classes = -1
            self.y_enc = self.y
            kf = KFold(n_splits=self.num_folds)
            train_prediction_shape = (lentrain, 1)

        self.train_prediction_dict = {}
        for level in range(self.levels):
            self.train_prediction_dict[level] = np.zeros((train_prediction_shape[0],
                                                          train_prediction_shape[1] * len(self.model_dict[level])))

        for level in range(self.levels):

            if level == 0:
                temp_train = self.training_data
            else:
                temp_train = self.train_prediction_dict[level - 1]

            for model_num, model in enumerate(self.model_dict[level]):
                validation_scores = []
                foldnum = 1
                for train_index, valid_index in kf.split(self.train_prediction_dict[0], self.y_enc):
logger.info("Training Level %d Fold # %d. Model # %d", level, foldnum, model_num)

                    if level != 0:
                        l_training_data = temp_train[train_index]
                        l_validation_data = temp_train[valid_index]
model.fit(l_training_data, self.y_enc[train_index])
                    else:
                        l0_training_data = temp_train[0][model_num]
                        if type(l0_training_data) == list:
                            l_training_data = [x[train_index] for x in l0_training_data]
                            l_validation_data = [x[valid_index] for x in l0_training_data]
                        else:
                            l_training_data = l0_training_data[train_index]
                            l_validation_data = l0_training_data[valid_index]
model.fit(l_training_data, self.y_enc[train_index])

logger.info("Predicting Level %d. Fold # %d. Model # %d", level, foldnum, model_num)

                    if self.task_type == 'classification':
                        temp_train_predictions = model.predict_proba(l_validation_data)
                        self.train_prediction_dict[level][valid_index,
                        (model_num * self.num_classes):(model_num * self.num_classes) +
                                                       self.num_classes] = temp_train_predictions

                    else:
                        temp_train_predictions = model.predict(l_validation_data)
                        self.train_prediction_dict[level][valid_index, model_num] = temp_train_predictions
                    validation_score = self.optimize(self.y_enc[valid_index], temp_train_predictions)
                    validation_scores.append(validation_score)
logger.info("Level %d. Fold # %d. Model # %d. Validation Score = %f", level, foldnum, model_num,
                                validation_score)
                    foldnum += 1
                avg_score = np.mean(validation_scores)
                std_score = np.std(validation_scores)
logger.info("Level %d. Model # %d. Mean Score = %f. Std Dev = %f", level, model_num,
                            avg_score, std_score)

logger.info("Saving predictions for level # %d", level)
            train_predictions_df = pd.DataFrame(self.train_prediction_dict[level])
            train_predictions_df.to_csv(os.path.join(self.save_path, "train_predictions_level_" + str(level) + ".csv"),
                                        index=False, header=None)

        return self.train_prediction_dict

    def predict(self, test_data, lentest):
        self.test_data = test_data
        if self.task_type == 'classification':
            test_prediction_shape = (lentest, self.num_classes)
        else:
            test_prediction_shape = (lentest, 1)

        self.test_prediction_dict = {}
        for level in range(self.levels):
            self.test_prediction_dict[level] = np.zeros((test_prediction_shape[0],
                                                         test_prediction_shape[1] * len(self.model_dict[level])))
        self.test_data = test_data
        for level in range(self.levels):
            if level == 0:
                temp_train = self.training_data
                temp_test = self.test_data
            else:
                temp_train = self.train_prediction_dict[level - 1]
                temp_test = self.test_prediction_dict[level - 1]

            for model_num, model in enumerate(self.model_dict[level]):

logger.info("Training Fulldata Level %d. Model # %d", level, model_num)
                if level == 0:
model.fit(temp_train[0][model_num], self.y_enc)
                else:
model.fit(temp_train, self.y_enc)

logger.info("Predicting Test Level %d. Model # %d", level, model_num)

                if self.task_type == 'classification':
                    if level == 0:
                        temp_test_predictions = model.predict_proba(temp_test[0][model_num])
                    else:
                        temp_test_predictions = model.predict_proba(temp_test)
                    self.test_prediction_dict[level][:, (model_num * self.num_classes): (model_num * self.num_classes) +
                                                                                        self.num_classes] = temp_test_predictions

                else:
                    if level == 0:
                        temp_test_predictions = model.predict(temp_test[0][model_num])
                    else:
                        temp_test_predictions = model.predict(temp_test)
                    self.test_prediction_dict[level][:, model_num] = temp_test_predictions

            test_predictions_df = pd.DataFrame(self.test_prediction_dict[level])
            test_predictions_df.to_csv(os.path.join(self.save_path, "test_predictions_level_" + str(level) + ".csv"),
                                       index=False, header=None)

        return self.test_prediction_dict



IndentationError: unindent does not match any outer indentation level (<tokenize>, line 15)

In [37]:
# specify the data to be used for every level of ensembling:
train_data_dict = {0: [xtrain_tfv, xtrain_ctv, xtrain_tfv, xtrain_ctv], 1: [xtrain_glove]}
test_data_dict = {0: [xvalid_tfv, xvalid_ctv, xvalid_tfv, xvalid_ctv], 1: [xvalid_glove]}

model_dict = {0: [LogisticRegression(), LogisticRegression(), MultinomialNB(alpha=0.1), MultinomialNB()],

              1: [xgb.XGBClassifier(silent=True, n_estimators=120, max_depth=7)]}

NameError: name 'xtrain_glove' is not defined

In [ ]:
ens = Ensembler(model_dict=model_dict, num_folds=3, task_type='classification',
                optimize=multiclass_logloss, lower_is_better=True, save_path='')
ens.fit(train_data_dict, ytrain, lentrain=xtrain_w2v.shape[0])
preds = ens.predict(test_data_dict, lentest=xvalid_w2v.shape[0])

In [ ]:
multiclass_logloss(yvalid, preds[1])